# 1. Import Module

In [29]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

# 2. Crawling Data

## 2.1. Function

In [17]:
def requestDataCTXH(mssv):
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en,vi;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://ctxh.uel.edu.vn/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.41',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Microsoft Edge";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        's': mssv.lower(),
    }

    response = requests.get('https://ctxh.uel.edu.vn/data.php', params=params, headers=headers)
    data = response.json()
    return data

In [18]:
def getNum(strNum):
    try:
        return float(strNum.replace(',', '.'))
    except:
        return 0

In [30]:
def SoluongNgayCTXH(mssv):
    try:
        data = requestDataCTXH(mssv)
        if len(data) == 0:
            return {'mssv': mssv, 'hoten': '', 'tongngay': -1}
        else:
            MSSV = data[0]['mssv'].upper() #check lại MSSV có trùng không
            HoTen = data[0]['hoten'] #check lại họ tên có trùng không
            tongNgay = 0.0
            for i in range(len(data)):
                tongNgay += getNum(data[i]['ngaythamgia'])
            return {'mssv': MSSV, 'hoten': HoTen, 'tongngay': tongNgay}
    except:
        return {'mssv': mssv, 'hoten': '', 'tongngay': -1}

In [31]:
SoluongNgayCTXH('K204110855')

{'mssv': 'K204110855', 'hoten': 'Đinh Văn An', 'tongngay': 15.5}

## 2.2. Crawling

In [26]:
#K20411 student id list
df = pd.read_csv('Data/k20411.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stt       67 non-null     int64 
 1   mssv      67 non-null     object
 2   ho        67 non-null     object
 3   ten       67 non-null     object
 4   ngaysinh  67 non-null     object
 5   lop       67 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.3+ KB


In [27]:
df.head()

,stt,mssv,ho,ten,ngaysinh,lop
0,1,K204110556,Nguyễn Thị Tâm,Anh,3/2/2002,K20411
1,2,K204110557,Từ Thị Ngọc,Anh,20/4/2002,K20411
2,3,K204110558,Vũ Thị Phương,Anh,8/2/2002,K20411
3,4,K204110559,Hoàng Ngọc Thảo,Duyên,28/10/2002,K20411
4,5,K204110560,Châu Võ Văn,Đạo,23/1/2002,K20411


In [37]:
df_result = pd.DataFrame(columns=['mssv', 'hoten', 'tongngay'])
df_result

,mssv,hoten,tongngay


In [39]:
for i in tqdm(range(len(df))):
    result = SoluongNgayCTXH(df['mssv'][i])
    df_result = df_result.append(result, ignore_index=True)

  0%|          | 0/67 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_16728\3102008805.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(result, ignore_index=True)
  1%|▏         | 1/67 [00:00<00:35,  1.86it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_16728\3102008805.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(result, ignore_index=True)
  3%|▎         | 2/67 [00:01<00:32,  2.02it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_16728\3102008805.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(result, ignore_index=True)
  4%|▍         | 3/67 [00:01<00:31,  2.03it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_16728\

In [40]:
df_result

,mssv,hoten,tongngay
0,K204110556,Nguyễn Thị Tâm Anh,7.5
1,K204110557,Từ Thị Ngọc Anh,18.5
2,K204110558,Vũ Thị Phương Anh,12.5
3,K204110559,Hoàng Ngọc Thảo Duyên,11.0
4,K204110560,Châu Võ Văn Đạo,17.5
...,...,...,...
62,K204111790,Phạm Ngọc Trâm,17.5
63,K204111791,Phạm Hồ Tiến Trung,22.0
64,K204111792,Thẩm Thị Tú Uyên,9.5
65,K204111793,Võ Ngọc Tường Vy,12.0


In [44]:
#merge data
df = pd.merge(df, df_result, on='mssv', how='left')
df

,stt,mssv,ho,ten,ngaysinh,lop,hoten_x,tongngay_x,hoten_y,tongngay_y
0,1,K204110556,Nguyễn Thị Tâm,Anh,3/2/2002,K20411,Nguyễn Thị Tâm Anh,7.5,Nguyễn Thị Tâm Anh,7.5
1,2,K204110557,Từ Thị Ngọc,Anh,20/4/2002,K20411,Từ Thị Ngọc Anh,18.5,Từ Thị Ngọc Anh,18.5
2,3,K204110558,Vũ Thị Phương,Anh,8/2/2002,K20411,Vũ Thị Phương Anh,12.5,Vũ Thị Phương Anh,12.5
3,4,K204110559,Hoàng Ngọc Thảo,Duyên,28/10/2002,K20411,Hoàng Ngọc Thảo Duyên,11.0,Hoàng Ngọc Thảo Duyên,11.0
4,5,K204110560,Châu Võ Văn,Đạo,23/1/2002,K20411,Châu Võ Văn Đạo,17.5,Châu Võ Văn Đạo,17.5
...,...,...,...,...,...,...,...,...,...,...
62,63,K204111790,Phạm Ngọc,Trâm,19/9/2002,K20411,Phạm Ngọc Trâm,17.5,Phạm Ngọc Trâm,17.5
63,64,K204111791,Phạm Hồ Tiến,Trung,27/11/2002,K20411,Phạm Hồ Tiến Trung,22.0,Phạm Hồ Tiến Trung,22.0
64,65,K204111792,Thẩm Thị Tú,Uyên,13/1/2002,K20411,Thẩm Thị Tú Uyên,9.5,Thẩm Thị Tú Uyên,9.5
65,66,K204111793,Võ Ngọc Tường,Vy,2/9/2002,K20411,Võ Ngọc Tường Vy,12.0,Võ Ngọc Tường Vy,12.0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 0 to 66
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stt       67 non-null     int64 
 1   mssv      67 non-null     object
 2   ho        67 non-null     object
 3   ten       67 non-null     object
 4   ngaysinh  67 non-null     object
 5   lop       67 non-null     object
 6   hoten     67 non-null     object
 7   tongngay  67 non-null     object
dtypes: int64(1), object(7)
memory usage: 4.7+ KB


In [43]:
df.to_csv('Data/k20411_ctxh.csv', index=False)